# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.06249


extracting tarball to tmp_2303.06249...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.06393


extracting tarball to tmp_2303.06393...

...

/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.07058/gj436_system.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'sections/appendix' from 'tmp_2303.07058/sections/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'sections/data_availability' from 'tmp_2303.07058/sections/data_availability.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/

extracting tarball to tmp_2303.07301...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.07318


extracting tarball to tmp_2303.07318...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.06650-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.06650) | **Signatures of winds and jets in the environment of supermassive black  holes**  |
|| Bidisha Bandyopadhyay, et al. -- incl., <mark>Christian Fendt</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *12 pages, 5 figures, Submitted to MNRAS*|
|**Abstract**| The Event Horizon Telescope Collaboration (EHTC) has presented first - dynamic-range limited - images of the black hole shadows in M87 and Sgr A*. The next generation Event Horizon Telescope (ngEHT) will provide higher sensitivity and higher dynamic range images (and movies) of these two sources plus image at least a dozen others at $\leq$100 gravitational radii resolution. We here perform an exploratory study of the appearance of winds and jets in such future observations. To do this we use M87 and Sgr A* as reference systems: we do not aim to exactly reproduce them, but rather to determine how their observed images will depend on specific physical assumptions. Even in the case of similar or the same dynamics, the images depend significantly on global parameters such as the black hole mass and the mass accretion rate. Our results provide guidance in the interpretation of future high-resolution images, particularly if a wind or jet is detected. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.06249-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.06249) | **A declining major merger fraction with redshift in the local Universe  from the largest-yet catalog of major and minor mergers in SDSS**  |
|| R. Nevin, et al. -- incl., <mark>J. A. Vázquez-Mata</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *29 pages, 22 figures, see figures 16 and 17 for a summary of the findings, accepted to MNRAS*|
|**Abstract**| It is difficult to accurately identify galaxy mergers and it is an even larger challenge to classify them by their mass ratio or merger stage. In previous work we used a suite of simulated mergers to create a classification technique that uses linear discriminant analysis (LDA) to identify major and minor mergers. Here, we apply this technique to 1.3 million galaxies from the SDSS DR16 photometric catalog and present the probability that each galaxy is a major or minor merger, splitting the classifications by merger stages (early, late, post-coalescence). We present publicly-available imaging predictor values and all of the above classifications for one of the largest-yet samples of galaxies. We measure the major and minor merger fraction ($f_{\mathrm{merg}}$) and build a mass-complete sample of galaxies, which we bin as a function of stellar mass and redshift. For the major mergers, we find a positive slope of $f_{\mathrm{merg}}$ with stellar mass and negative slope of $f_{\mathrm{merg}}$ with redshift between stellar masses of $10.5 < M_* (log\ M_{\odot}) < 11.6$ and redshifts of $0.03 < z < 0.19$. We are able to reproduce an artificial positive slope of the major merger fraction with redshift when we do not bin for mass or craft a complete sample, demonstrating the importance of mass completeness and mass binning. We determine that the positive trend of the major merger fraction with stellar mass is consistent with a hierarchical assembly scenario. The negative trend with redshift requires that an additional assembly mechanism, such as baryonic feedback, dominates in the local Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.06733-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.06733) | **Probing the Origin of Changing-look Quasar Transitions with Chandra**  |
|| Qian Yang, et al. -- incl., <mark>Paul J. Green</mark>, <mark>Xiurui Zhao</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *26 pages, 9 figures. To be submitted to ApJ*|
|**Abstract**| Extremely variable quasars can also show strong changes in broad line emission strength, and are known as Changing Look Quasars (CLQs). To study the CLQ transition mechanism, we present a pilot sample of CLQs with X-ray observations both in the bright and faint states. From a sample of quasars with bright state archival SDSS spectra and (Chandra or XMM-Newton) X-ray data, we identified five new CLQs via optical spectroscopic follow-up, and then obtained new target-of-opportunity X-ray observations with Chandra. No strong absorption is detected in either the bright or the faint state X-ray spectra. The intrinsic X-ray flux generally tracks the optical variability, and the X-ray power-law slope becomes harder in the faint state. Large amplitude mid-infrared variability is detected in all five CLQs. The changing-obscuration model is not consistent with the observed X-ray spectra and spectral energy distribution changes seen in these CLQs. It is highly likely that the observed changes are due to the changing accretion rate of the supermassive black hole, so the multiwavelength emission varies accordingly, with promising analogies to the accretion states of X-ray binaries. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07058-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07058) | **The energetic particle environment of a GJ 436 b-like planet**  |
|| <mark>D. Rodgers-Lee</mark>, et al. |
|*Appeared on*| *2023-03-14*|
|*Comments*| *14 pages, 9 figures, accepted for publication in MNRAS*|
|**Abstract**| A key first step to constrain the impact of energetic particles in exoplanet atmospheres is to detect the chemical signature of ionisation due to stellar energetic particles and Galactic cosmic rays. We focus on GJ$\,$436, a well-studied M dwarf with a warm Neptune-like exoplanet. We demonstrate how the maximum stellar energetic particle momentum can be estimated from the stellar X-ray luminosity. We model energetic particle transport through the atmosphere of a hypothetical exoplanet at orbital distances between $a=0.01-0.2\,$au from GJ$\,$436, including GJ$\,$436$\,$b's orbital distance (0.028$\,$au). For these distances we find that, at top-of-atmosphere, stellar energetic particles ionise molecular hydrogen at a rate of $\zeta_{\rm StEP,H_2} \sim 4\times10^{-10}-2\times10^{-13}\,\mathrm{s^{-1}}$. In comparison, Galactic cosmic rays alone lead to $\zeta_{\rm GCR, H_2}\sim2\times 10^{-20}-10^{-18} \,\mathrm{s^{-1}}$. At 10au we find that ionisation due to Galactic cosmic rays equals that of stellar energetic particles: $\zeta_{\rm GCR,H_2} = \zeta_{\rm StEP,H_2} \sim 7\times10^{-18}\,\rm{s^{-1}}$ for the top-of-atmosphere ionisation rate. At GJ$\,$436$\,$b's orbital distance, the maximum ion-pair production rate due to stellar energetic particles occurs at pressure $P\sim 10^{-3}\,$bar while Galactic cosmic rays dominate for $P>10^2\,$bar. These high pressures are similar to what is expected for a post-impact early Earth atmosphere. The results presented here will be used to quantify the chemical signatures of energetic particles in warm Neptune-like atmospheres. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07301-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07301) | **Phantom dark energy as a natural selection of evolutionary processes  $\hat{\rm a}$ $\textit{la}$ $\textit{genetic algorithm}$ and cosmological  tensions**  |
|| Mayukh R. Gangopadhyay, M. Sami, <mark>Mohit K. Sharma</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *5 pages, 4 figures*|
|**Abstract**| We employ a machine learning (ML) algorithm to analyze cosmological background data and the linear red-shift space distortion (RSD) data in a model-independent way, with specific focus on the Hubble expansion rate and the growth of large-scale structure. We find strong evidence that the natural enhancement in the Hubble parameter at low redshifts is due to the underlying phantom nature of dark energy, rather than low matter density. As for the RSD data, we find a higher value of $\sigma^8_{(0)}$ which is consistent with CMB's predictions, but the outcome of low matter density leads to unresolved tension. This might point towards a new physical phenomenon at the perturbative level in the low redshift regime. From a statistical perspective, we have demonstrated that our results hold greater preference compared to those obtained by the $\Lambda$CDM model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.07318-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.07318) | **The Masses of Supernova Remnant Progenitors in M33**  |
|| Brad Koplitz, et al. -- incl., <mark>Joseph Guzman</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *20 pages, 7 figures, 2 tables*|
|**Abstract**| Using resolved optical stellar photometry from the Panchromatic Hubble Andromeda Treasury Triangulum Extended Region (PHATTER) survey, we measured the star formation history (SFH) near the position of 85 supernova remnants (SNRs) in M33. We constrained the progenitor masses for 60 of these SNRs, finding the remaining 25 remnants had no local SF in the last 56 Myr consistent with core-collapse SNe (CCSNe), making them potential Type Ia candidates. We then infer a progenitor mass distribution from the age distribution, assuming single star evolution. We find that the progenitor mass distribution is consistent with being drawn from a power-law with an index of $-2.9^{+1.2}_{-1.0}$. Additionally, we infer a minimum progenitor mass of $7.1^{+0.1}_{-0.2}$ $M_{\odot}$ from this sample, consistent with several previous studies, providing further evidence that stars with ages older than the lifetimes of single 8 $M_{\odot}$ stars are producing supernovae. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.06592-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.06592) | **Spatially Resolving PAHs in Herbig Ae Disks with VISIR-NEAR at the VLT**  |
|| Gideon Yoffe, et al. -- incl., <mark>Roy van Boekel</mark>, <mark>M.R Meyer</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| **|
|**Abstract**| We use the long-slit spectroscopy mode of the VISIR-NEAR experiment to perform diffraction-limited observations of eight nearby Herbig Ae protoplanetary disks. We extract spectra for various locations along the slit with a spectral resolution of R = 300 and perform a compositional fit at each spatial location using spectral templates of silicates and the four PAH bands. This yields the intensity vs. location profiles of each species. Results. We could obtain spatially-resolved intensity profiles of the PAH emission features in the N-band for five objects (AB Aurigae, HD 97048, HD 100546, HD 163296, and HD 169142). We observe two kinds of PAH emission geometry in our sample: centrally-peaked (HD 97048) and ring-like (AB Aurigae, HD 100546, HD 163296, and potentially HD 169142). Comparing the spatial PAH emission profiles with near-infrared scattered light images, we find a strong correlation in the disk sub-structure but a difference in radial intensity decay rate. The PAH emission shows a less steep decline with distance from the star. Finally, we find a correlation between the presence of (sub-) micron-sized silicate grains leading to the depletion of PAH emission within the inner regions of the disks. In this work, we find the following: (1) PAH emission traces the extent of Herbig Ae disks to a considerable radial distance. (2) The correlation between silicate emission within the inner regions of disks and the depletion of PAH emission can result from dust-mixing and PAH coagulation mechanisms and competition over UV photons. (3) For all objects in our sample, PAHs undergo stochastic heating across the entire spatial extent of the disk and are not saturated. (4) The difference in radial intensity decay rates between the PAHs and scattered-light profiles may be attributed to shadowing and dust-settling effects, which affect the scattering grains more than the PAHs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.06453-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.06453) | **Evidence of a radiation belt around a brown dwarf**  |
|| J. B. Climent, et al. -- incl., <mark>M. Pérez-Torres</mark> |
|*Appeared on*| *2023-03-14*|
|*Comments*| *Submitted on 12 January 2023 to Science. In review*|
|**Abstract**| Radio observations of ultracool dwarfs, objects comprising brown dwarfs and the very lowest mass stars, have mainly focused on analyzing their light-curve and spectral energy distributions providing valuable insights into their magnetic fields. However, spatially-resolved studies of such magnetospheres have been elusive so far. Radio interferometric observations of the brown dwarf LSR J1835+3259 reveal an extended magnetosphere with a morphology compatible with the presence of a radiation belt, similar to that of Jupiter and Earth, consisting of energetic particles confined via magnetic mirroring. Our finding suggests that radio emitting ultracool dwarfs may behave as scaled up versions of Jupiter, validating the connection between dipole-ordered magnetic fields and the presence of belt-like morphologies and aurorae beyond our Solar System. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.06650.md
    + _build/html/tmp_2303.06650/./smudgethallsimM87SgrAi17f230EHT.png
    + _build/html/tmp_2303.06650/./prothM87SgrAallsimf230i17EHT.png
    + _build/html/tmp_2303.06650/./smudgethallsimM87SgrAi17f230GEOVLBI.png
    + _build/html/tmp_2303.06650/./prothM87SgrAallsimf230i17GEOVLBI.png
    + _build/html/tmp_2303.06650/./smudgethallsimM87SgrAi17f230L1VLBI.png
    + _build/html/tmp_2303.06650/./prothM87SgrAallsimf230i17L1VLBI.png
    + _build/html/tmp_2303.06650/./allsimi17M87SgrAf345thR12NDif.png
    + _build/html/tmp_2303.06650/./allsimi17M87SgrAf700thR12NDif.png
    + _build/html/tmp_2303.06650/./prothM87SgrAi17allsimf345NDif.png
    + _build/html/tmp_2303.06650/./prothM87SgrAi17allsimf700NDif.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Signatures of winds and jets in the environment of supermassive black holes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.06650-b31b1b.svg)](https://arxiv.org/abs/2303.06650)<mark>Appeared on: 2023-03-14</mark> - _12 pages, 5 figures, Submitted to MNRAS_

</div>
<div id="authors">

Bidisha Bandyopadhyay, et al. -- incl., <mark><mark>Christian Fendt</mark></mark>

</div>
<div id="abstract">

**Abstract:** The Event Horizon Telescope Collaboration (EHTC) has presented first - dynamic-range limited - images of the black hole shadows in M87 and Sgr A*. The next generation Event Horizon Telescope (ngEHT) will provide higher  sensitivity and higher dynamic range images (and movies) of these two sources plus image at least a dozen others at $\leq$ 100 gravitational radii resolution. We here perform an exploratory study of the appearance of winds and jets in such future observations. To do this we use  M87 and Sgr A* as reference systems: we do not aim to exactly reproduce them, but rather to determine how their observed images will depend on specific physical assumptions. Even in the case of similar or the same dynamics, the images depend significantly on global parameters such as the black hole mass and the mass accretion rate. Our results provide guidance in the interpretation of future high-resolution images, particularly if a wind or jet is detected.

</div>

<div id="div_fig1">

<img src="tmp_2303.06650/./smudgethallsimM87SgrAi17f230EHT.png" alt="Fig1.1" width="16%"/><img src="tmp_2303.06650/./prothM87SgrAallsimf230i17EHT.png" alt="Fig1.2" width="16%"/><img src="tmp_2303.06650/./smudgethallsimM87SgrAi17f230GEOVLBI.png" alt="Fig1.3" width="16%"/><img src="tmp_2303.06650/./prothM87SgrAallsimf230i17GEOVLBI.png" alt="Fig1.4" width="16%"/><img src="tmp_2303.06650/./smudgethallsimM87SgrAi17f230L1VLBI.png" alt="Fig1.5" width="16%"/><img src="tmp_2303.06650/./prothM87SgrAallsimf230i17L1VLBI.png" alt="Fig1.6" width="16%"/>

**Figure 1. -** Emission maps and radial emission profiles (along diameters with angles$=0◦, 90◦, 45◦ \& -45◦ $ respectively) for M87 (upper 6 in each panel) and SgrA* (lower 6 in each panel) like systems for inclination angles of $17◦$ at 230 GHz for EHT-$20 \mu$arcsec (upper), Geo-VLBI-$5 \mu$arcsec (middle) and L1-VLBI-$0.16 \mu$arcsec (lower) like resolutions). The intensity profiles in addition also mark the positions of the unlensed photon ring (black solid line) and the theoretical values of the inner (dot dashed line) and outer (dashed line) boundaries of the lensed photon ring. (*fig:M87SgrAL1VLBI*)

</div>
<div id="div_fig2">

<img src="tmp_2303.06650/./allsimi17M87SgrAf345thR12NDif.png" alt="Fig2.1" width="25%"/><img src="tmp_2303.06650/./allsimi17M87SgrAf700thR12NDif.png" alt="Fig2.2" width="25%"/><img src="tmp_2303.06650/./prothM87SgrAi17allsimf345NDif.png" alt="Fig2.3" width="25%"/><img src="tmp_2303.06650/./prothM87SgrAi17allsimf700NDif.png" alt="Fig2.4" width="25%"/>

**Figure 2. -** Flux normalized emission maps (upper) and intensity profiles (lower) (along diameters with angles$=0◦, 90◦, 45◦ \& -45◦ $ respectively) for all our simulations for M87 (upper 6 in each of the panels) and and SgrA* (lower 6 in each of the panels) like systems with inclination angle $17◦$ for 345 GHz (left) and 700 GHz (right) respectively. The intensity profiles in addition also mark the positions of the unlensed photon ring (black solid line) and the theoretical values of the inner (dot dashed line) and outer (dashed line) boundaries of the lensed photon ring. (*fig:M87SgrAang345n700*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

196  publications in the last 7 days.
	 _build/html/2303.06650.md
	 _build/html/2303.05635.md
	 _build/html/2303.04727.md
	 _build/html/2303.04067.md
	 _build/html/2303.03420.md
	 _build/html/2303.02816.md
	 _build/html/2303.01528.md
	 _build/html/2303.00044.md
	 _build/html/2303.00012.md
	 _build/html/2302.14137.md
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
print(carousel, docs, slides)
print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.
  <div class="carousel" 
       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">
    <div class="carousel-cell"> <div id="slide1" class="md_view">Content 1</div> </div>
    <div class="carousel-cell"> <div id="slide2" class="md_view">Content 2</div> </div>
  </div> "2303.06650.md", "2303.05635.md" "slide1", "slide2"
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <!-- Remove caching as much as possible -->
    <meta http-equiv="cache-control" content="no-cache" />
    <meta http-equiv="Pragma" content="no-cache" />
    <meta http-equiv="Expires" content="-1" />
    <!-- flickity bootstrap CSS -->
    <link rel="stylesheet" href="https://unpkg.com/flickity@2/dist/flickity.min.css">
    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdeli